In [25]:
import os
import json
import openai
import asyncio
import tiktoken

from bs4 import BeautifulSoup
from pymongo import MongoClient
from scrapy.selector import Selector
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import LLMExtractionStrategy

In [26]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Connect to db, etc

In [27]:
# Connect to the MongoDB server
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client['crawler']  # Replace with your database name
cache_coll = db['httpcache']
batches_coll = db['batches']  # Replace with your collection name
urls_coll = db['urls_db'] 
content_coll = db['content'] 

In [28]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens



### Strategy 1

In [29]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        response_format={ "type": "json_object" },
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [30]:
query = {"fingerprint": '4f834bcc47a545cb55894d101761bf5ee80684e0'}
projection = {'url': 1, 'body': 1, '_id': 0}
documents = list(content_coll.find(query, projection))

doc = documents[0]

len(documents)

1

In [31]:
doc['url']

'https://hotnews.ro/cat-de-periculoasa-ar-fi-victoria-lui-georgescu-si-cat-ar-dura-iesirea-romaniei-din-ue-daca-independentul-ar-ajunge-la-cotroceni-explicatiile-unui-vicepresedinte-al-parlamentului-european-1847441'

In [32]:
doc

{'body': '<body class="post-template-default single single-post postid-1847441 single-format-standard wp-embed-responsive cat-de-periculoasa-ar-fi-victoria-lui-georgescu-si-cat-ar-dura-iesirea-romaniei-din-ue-daca-independentul-ar-ajunge-la-cotroceni-explicatiile-unui-vicepresedinte-al-parlamentului-european-1847441 category-actualitate cat-color-blue">\n    \n    <div id="app" class="site">\n      <a class="sr-only focus:not-sr-only" href="#main">\n        Sari direct la conținut\n      </a>\n\n      <header id="js-site-header" class="site-header">\n      <aside class="row white curs-valutar" aria-labelledby="curs-valutar">\n        <div class="hn-container">\n            <div class="box">\n                <div class="last-updated">\n                    <img src="https://hotnews.ro/wp-content/themes/hotnews/public/images/ic_updated.svg" class="cv-icon" alt="cv-icon">\n                    <span class="label-text"> <span class="break">Actualizat</span> acum 15 minute</span>\n           

In [33]:
body = doc['body']
soup = BeautifulSoup(body, "html.parser")

type(body), len(body), len(soup.prettify())

(str, 182725, 189004)

In [34]:
print(len(soup.prettify()))

for tag in soup(['script', 'style', 'meta', 'head', 'nav', 'footer', 'aside', 'svg']):
    tag.decompose()

print(len(soup.prettify()))

189004
59397


In [35]:
soup.prettify()

'<body class="post-template-default single single-post postid-1847441 single-format-standard wp-embed-responsive cat-de-periculoasa-ar-fi-victoria-lui-georgescu-si-cat-ar-dura-iesirea-romaniei-din-ue-daca-independentul-ar-ajunge-la-cotroceni-explicatiile-unui-vicepresedinte-al-parlamentului-european-1847441 category-actualitate cat-color-blue">\n <div class="site" id="app">\n  <a class="sr-only focus:not-sr-only" href="#main">\n   Sari direct la conținut\n  </a>\n  <header class="site-header" id="js-site-header">\n   <div class="row blue main-area">\n    <div class="hn-container">\n     <div class="main-row">\n      <div class="box">\n       <a href="https://hotnews.ro/">\n        <img alt="HotNews.ro" class="logo" src="https://hotnews.ro/wp-content/themes/hotnews/public/images/hotnews_header.svg"/>\n       </a>\n      </div>\n      <div class="box">\n       <div class="menu-items">\n        <div class="menu-btn" id="js-trigger-search-form">\n         <img alt="search_icon" class="icon

In [36]:
text = soup.get_text(separator=' ', strip=True)

len(text), num_tokens_from_string(text, 'gpt-4o')

(11267, 3324)

In [37]:
text

'Sari direct la conținut CAUTĂ Actualitate joi, 28 noiembrie 2024, 00:19 Cât de periculoasă ar fi victoria lui Georgescu și cât ar dura ieşirea României din UE dacă independentul ar ajunge la Cotroceni – explicațiile unui vicepreşedinte al Parlamentului European Nicoleta Onofrei HotNews.ro Share Vicepreşedintele Parlamentului European, Nicu Ştefănuţă, explică, miercuri seară, cât de periculoasă ar fi victoria lui Călin Georgescu pentru politica externă a României, dar și cât ar dura o un ROEXIT și o eventuală ieșire din NATO, în cazul în care acesta ar ajunge la Cotroceni și ar vrea să îndepărteze țara noastră de alianța transatlantică și de blocul comunitar european, subliniind că preşedintele unei ţări nu poate deveni dictator peste noapte. În contextul în care Călinescu a declarat că „NATO este cea mai slabă alianță de pe pământ” și „dacă lucrurile continuă în felul acesta”, ar abandona-o instant, vicepreşedintele Parlamentului European Nicu Ștefănuță a făcut câteva precezări, pentr

#### Strategy no 1:
* clean html file of polutting elemnents, 'script', 'style', 'meta', 'head', 'nav', 'footer', 'aside'
* extract all text. at this point we should have the article text, title, date, etc we need but also some garbage from other element
* prompt llm to sieve out the garbage and keep only the relevant info

##### Features
* low input token count per page ~3-4 tokens -> lowihs cost, fast
* higher output token count similar to input bc it return the same text. could try asking to return indices
* gpt-4o-mini appears to handle the job well (cheaper)
* could batch a lot of articles in the same prompt considering 'gpt-4o-mini' 128k context window (untested!)
* its actually about 4X as originally thought. output tokens have a 4x higher cost and output is just a slightly trimmed down version of the input
* consider: 1) training a smaller (bert-like?) model instead with data generated from llms (start-stop indices) or 2) since the text would already go through proccesing request outputs of text processing tasks

In [38]:
prompt = f"""the following text is a news article scraped off a news webpage and is polluted with bits of text that are not related to the subject matter of the article.
determine the relevant data and leave out any unrelated data.
return a json object with the article date, title and body
the text is bellow:
{text}"""
model = 'gpt-4o-mini'
res = get_completion(prompt, model=model)

print(res)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "date": "2024-11-28",
  "title": "Cât de periculoasă ar fi victoria lui Georgescu pentru politica externă a României",
  "body": "Vicepreşedintele Parlamentului European, Nicu Ştefănuţă, explică, miercuri seară, cât de periculoasă ar fi victoria lui Călin Georgescu pentru politica externă a României, dar și cât ar dura o un ROEXIT și o eventuală ieșire din NATO, în cazul în care acesta ar ajunge la Cotroceni și ar vrea să îndepărteze țara noastră de alianța transatlantică și de blocul comunitar european, subliniind că preşedintele unei ţări nu poate deveni dictator peste noapte. În contextul în care Călinescu a declarat că „NATO este cea mai slabă alianță de pe pământ” și „dacă lucrurile continuă în felul acesta”, ar abandona-o instant, vicepreşedintele Parlamentului European Nicu Ștefănuță a făcut câteva precezări, pentru că „cu cât suntem mai bine informaţi, cu atât reducem anxietatea şi putem comunica mai eficient, mai clar, unii cu ceilalţi”. Analiza declarațiilor lui Călin Geo

### Strategy 2

### Automate xpath selectors


* clean html file of polutting elemnents, 'script', 'style', 'meta', 'head', 'nav', 'footer', 'aside'
* extract all text into a list of numbered dicts with item, text, and css_classes keys
* convert that to json
* 
* prompt llm to sieve out the garbage and keep only the relevant info
* ask to return xpath selectors for relevant fields
* run once a day for each website and use selectors in Scrapy spiders

##### Features
* 

In [39]:
elements = []

def get_xpath(tag):
    path = []
    while tag is not None and tag.name != '[document]':
        siblings = tag.find_previous_siblings(tag.name)  # Get previous siblings of the same type
        index = len(siblings) + 1  # XPath indices are 1-based
        path.insert(0, f"{tag.name}[{index}]")
        tag = tag.parent
    return "/" + "/".join(path)

def get_xpath(tag):
    path = []
    while tag is not None and tag.name != '[document]':
        siblings = tag.find_previous_siblings(tag.name)
        index = len(siblings) + 1  # XPath indices are 1-based
        class_attr = ".".join(tag.get("class", []))  # Combine CSS classes
        if class_attr:
            path.insert(0, f"{tag.name}[{index}][class='{class_attr}']")
        else:
            path.insert(0, f"{tag.name}[{index}]")
        tag = tag.parent
    return "/" + "/".join(path)

def get_xpath(tag):
    path = []
    while tag is not None and tag.name != '[document]':
        classes = [c for c in tag.get("class", []) if len(c) < 20]
        class_attr = ".".join(classes)  # Combine CSS classes
        if class_attr:
            # if len(class_attr) < 25:
            path.insert(0, f"{tag.name}[class='{class_attr}']")
        else:
            path.insert(0, tag.name)
        tag = tag.parent
    return "/" + "/".join(path)

# Remove non-visible elements like <script> and <style>
for tag in soup(['script', 'style']):
    tag.decompose()

for idx, tag in enumerate(soup.find_all(), start=1):
    # Direct text only: subtract nested children's text
    direct_text = ''.join(tag.find_all(text=True, recursive=False)).strip()
    if direct_text:  # Only include tags with direct text
        elements.append({
            "id": idx,  # Add numbering
            "item": tag.name,  # Tag name
            "text": direct_text,  # Direct text only
            "css_classes": tag.get("class", []),  # CSS classes as a list
            "xpath": get_xpath(tag)  # Generate XPath-like selector
        })

text = json.dumps(elements, indent=4, ensure_ascii=False)

C:\Users\Asus\AppData\Local\Temp\ipykernel_16864\727659395.py:44: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  direct_text = ''.join(tag.find_all(text=True, recursive=False)).strip()


In [40]:
len(elements)

72

In [41]:
elements

[{'id': 2,
  'item': 'div',
  'text': '#colophon',
  'css_classes': ['site'],
  'xpath': "/body[class='single.single-post.postid-1847441.wp-embed-responsive.cat-color-blue']/div[class='site']"},
 {'id': 3,
  'item': 'a',
  'text': 'Sari direct la conținut',
  'css_classes': ['sr-only', 'focus:not-sr-only'],
  'xpath': "/body[class='single.single-post.postid-1847441.wp-embed-responsive.cat-color-blue']/div[class='site']/a[class='sr-only.focus:not-sr-only']"},
 {'id': 24,
  'item': 'span',
  'text': 'CAUTĂ',
  'css_classes': [],
  'xpath': "/body[class='single.single-post.postid-1847441.wp-embed-responsive.cat-color-blue']/div[class='site']/header[class='site-header']/div[class='row.blue.main-area']/div[class='hn-container']/div[class='search-form']/form[class='form']/button[class='go-search']/span"},
 {'id': 35,
  'item': 'article',
  'text': '.entry-header',
  'css_classes': ['article-single-page',
   'cat-color-blue',
   'post-1847441',
   'post',
   'type-post',
   'status-publish',


In [42]:
prompt = f""" you are looking at a json of html elements scraped of a news website

the following text is a news article scraped off a news webpage and is polluted with bits of text that are not related to the subject matter of the article.
determine the relevant data and leave out any unrelated data.

return a json object with the article date, title, author and article body each with the list of element ids relevant for it as well as the actual text

return also a json with a simple, minimal, short, xpath selector that will extract the text for the categories described above
prefer using css classes over order selectors
end the xpath with text()


json is bellow:
{text}"""
model = 'gpt-4o-mini'
res = get_completion(prompt, model=model)

# for the article body return only indices for the first and last words but not the article body itself

print(res)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "article": {
    "date": {
      "element_ids": [43],
      "text": "joi, 28 noiembrie 2024, 00:19"
    },
    "title": {
      "element_ids": [44],
      "text": "Cât de periculoasă ar fi victoria lui Georgescu și cât ar dura ieşirea României din UE dacă independentul ar ajunge la Cotroceni – explicațiile unui vicepreşedinte al Parlamentului European"
    },
    "author": {
      "element_ids": [48],
      "text": "Nicoleta Onofrei"
    },
    "body": {
      "element_ids": [
        72, 73, 74, 75, 79, 81, 82, 83, 85, 86, 87, 89, 90, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 105, 106, 112, 113, 114, 120, 138, 139, 141, 145, 152, 159, 166, 173, 180, 187, 194, 201, 208, 215, 222, 229, 236, 243, 250, 257, 264, 271, 278, 285, 292, 299, 306, 313, 320, 327, 334, 341
      ],
      "text": "Vicepreşedintele Parlamentului European, Nicu Ştefănuţă, explică, miercuri seară, cât de periculoasă ar fi victoria lui Călin Georgescu pentru politica externă a României, dar și cât ar dura o un RO

In [49]:
prompt = f""" you are looking at a json of html elements scraped of a news website

the following text is a news article scraped off a news webpage and is polluted with bits of text that are not related to the subject matter of the article.
determine the relevant data and leave out any unrelated data.

return a json object with the article date, title, author and article body each with the list of element ids relevant for it as well as the actual text

return also a json with a simple, minimal, short, xpath selector that will extract the text for the categories described above
prefer using css classes over order selectors
end the xpath with text()


json is bellow:
{text}"""
model = 'gpt-4o'
res = get_completion(prompt, model=model)

# for the article body return only indices for the first and last words but not the article body itself

print(res)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



{
  "article": {
    "date": {
      "element_ids": [43],
      "text": "joi, 28 noiembrie 2024, 00:19"
    },
    "title": {
      "element_ids": [44],
      "text": "Cât de periculoasă ar fi victoria lui Georgescu și cât ar dura ieşirea României din UE dacă independentul ar ajunge la Cotroceni – explicațiile unui vicepreşedinte al Parlamentului European"
    },
    "author": {
      "element_ids": [48],
      "text": "Nicoleta Onofrei"
    },
    "body": {
      "element_ids": [72, 73, 75, 81, 82, 83, 85, 87, 89, 90, 92, 93, 95, 96, 97, 98, 100, 102, 103, 105, 106],
      "text": "Vicepreşedintele Parlamentului European, Nicu Ştefănuţă, explică, miercuri seară, cât de periculoasă ar fi victoria lui Călin Georgescu pentru politica externă a României, dar și cât ar dura o un ROEXIT și o eventuală ieșire din NATO, în cazul în care acesta ar ajunge la Cotroceni și ar vrea să îndepărteze țara noastră de alianța transatlantică și de blocul comunitar european, subliniind că preşedintele un

In [43]:
len(prompt), len(res)

(54822, 5709)

In [44]:
from parsel import Selector

In [45]:
sel = Selector(doc['body'])

In [46]:
body = sel.xpath("//time[@class='entry-date published']").extract()[0]

In [47]:
sel.xpath("//a[@class='author vcard']//text()").getall()
sel.xpath("//body/div[@class='site']//a[contains(@class, 'author.vcard')]//text()").getall()

[]

In [48]:
sel.xpath("//div[@class='entry-content']//text()").getall()
sel.xpath("//body/div[@class='site']//div[@class='entry-content']/text()").getall()

['\n    \n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n',
 '\n  ']